# Tutorial 2: Introduction to Gridap.jl - Integration of elemental matrix and vector

Welcome to the second tutorial in the CMOE series on computational methods for multiphysics problems! In this tutorial, you will explore **the numerical integration process** that transforms weak formulations into elemental matrices and vectors in Gridap.jl.

> **📘 About this tutorial**: While Tutorial 1 covered the finite element workflow from weak forms to elemental matrix calculation, this tutorial focuses on what happens **inside the integration step**. We'll dissect the `integrate` function, examine quadrature rules, understand reference vs physical domain evaluations, and explore Gridap's lazy evaluation strategies. This is essential for understanding how Gridap achieves both mathematical elegance and computational efficiency.

> **⚠️ Note**: The text in this tutorial has been generated by LLMs. If you find any inconsistencies, please report them through an issue in the repository.

## Table of Contents

**Part 1: Integration Basics**
- 1.1 [Setting Up the Problem](#11-setting-up-the-problem)
- 1.2 [Obtaining Basis Functions and Domain Contributions](#12-obtaining-basis-functions-and-domain-contributions)
- 1.3 [Understanding the `integrate` Function](#13-understanding-the-integrate-function)

**Part 2: Quadrature and Evaluation**
- 2.1 [Quadrature Points and Cell Data Structures](#21-quadrature-points-and-cell-data-structures)
- 2.2 [Evaluating Weak Form Objects at Quadrature Points](#22-evaluating-weak-form-objects-at-quadrature-points)
- 2.3 [Domain Styles: Reference vs Physical Integration](#23-domain-styles-reference-vs-physical-integration)

**Part 3: Jacobian Transformations and Integration Maps**
- 3.1 [Cell Maps and Jacobian Computation](#31-cell-maps-and-jacobian-computation)
- 3.2 [The Integration Map: Combining Weights and Jacobians](#32-the-integration-map-combining-weights-and-jacobians)
- 3.3 [Lazy Evaluation Strategy in Gridap](#33-lazy-evaluation-strategy-in-gridap)

## Learning Objectives

By the end of this tutorial, you will be able to:
- Understand how the `integrate` function works internally in Gridap
- Extract and inspect quadrature points and weights for numerical integration
- Distinguish between reference domain and physical domain operations
- Compute Jacobian transformations for isoparametric mappings
- Understand Gridap's lazy evaluation strategy using `lazy_map`
- Manually replicate the integration process step-by-step
- Recognize how domain styles affect computation efficiency
- Debug integration-related issues by inspecting intermediate steps

This tutorial reveals the computational machinery beneath Gridap's elegant syntax. Understanding these internals will help you implement custom integration schemes, optimize performance, and troubleshoot complex problems.

## Conceptual Overview

The numerical integration process in finite element methods follows this pipeline:

```
Weak Form Expression
     ↓
∫(∇u·∇v)dΩ  ← Mathematical notation
     ↓
[Discretization with basis functions]
     ↓
∫(∇φⱼ·∇ψᵢ)dΩ  ← Element-level integrals
     ↓
[Numerical quadrature]
     ↓
Σ (∇φⱼ·∇ψᵢ)(xq)·wq·|J(ξq)|  ← Sum over quadrature points
     ↓
Kᵉᵢⱼ  ← Elemental matrix entries
```

**Key Steps We'll Explore**:

1. **Quadrature Point Generation**: Where and why integration points are chosen
2. **Function Evaluation**: Computing basis function values/gradients at quadrature points
3. **Reference vs Physical Domain**: Understanding coordinate transformations
4. **Jacobian Computation**: Accounting for geometry in the integration measure
5. **Weighted Summation**: Combining quadrature weights with integrand values
6. **Lazy Evaluation**: How Gridap delays computation for efficiency

## Problem Context

We continue with the **Poisson equation** from Tutorial 1:

$$-\Delta u = f \quad \text{in } \Omega$$

But now we focus on the integration aspect:

$$K^e_{ij} = \int_{\Omega_e} \nabla \psi_i \cdot \nabla \phi_j \, d\Omega \approx \sum_{q=1}^{n_q} (\nabla \psi_i \cdot \nabla \phi_j)(\mathbf{x}_q) \, w_q \, |\mathbf{J}(\boldsymbol{\xi}_q)|$$

**Questions We'll Answer**:
- How does Gridap choose quadrature points $\mathbf{x}_q$?
- Where do the weights $w_q$ come from?
- How is the Jacobian $|\mathbf{J}|$ computed?
- What's the difference between reference and physical domain evaluation?
- How does lazy evaluation improve performance?

## Prerequisites

- Completion of **Tutorial 1** (or familiarity with basic Gridap workflow)
- Understanding of numerical integration concepts (Gauss quadrature)
- Knowledge of reference element transformations in FEM
- Familiarity with Julia's functional programming features (anonymous functions, `map`)
- Basic understanding of the chain rule for coordinate transformations

## What Makes This Tutorial Different

While Tutorial 1 showed you **what** Gridap does, this tutorial shows you **how** it does it. You'll:
- Look inside the `integrate` function
- Manually reconstruct integration steps
- Understand Gridap's performance optimizations
- Gain tools for debugging and customization

Let's dive into the computational core of finite element integration!

---

# Part 1: Integration Basics

In this first part, we'll set up the problem (reusing components from Tutorial 1) and begin exploring the integration machinery. We'll see how evaluating weak forms with basis functions produces domain contributions, and how the `integrate` function transforms these into elemental matrices.

---

## 1.1 Setting Up the Problem

Before diving into integration details, we need to set up the same finite element problem from Tutorial 1. This provides the foundation for exploring the integration process.

### Activating the Project Environment

As in Tutorial 1, we start by activating the Julia environment:

```julia
Pkg.activate("..")
Pkg.instantiate()
```

This ensures:
- **Reproducibility**: Same package versions as other tutorials
- **Consistency**: All dependencies properly resolved
- **Gridap availability**: The finite element library is loaded

For a detailed explanation of environment activation, see **Tutorial 1, Section 1.1**.

### Loading Gridap

```julia
using Gridap
```

This single command imports all necessary Gridap functionality, including:
- Mesh generation tools
- Finite element space definitions
- Integration measures and quadrature
- Assembly routines
- The internal functions we'll explore in this tutorial

### Quick Problem Setup

We'll use the same $2 \times 2$ mesh and Poisson problem from Tutorial 1:

**Geometry**:
- Domain: $[0, 2] \times [0, 2]$ square
- Mesh: $2 \times 2$ = 4 quadrilateral cells
- Element size: $h = 1.0 \times 1.0$

**PDE**:
$$-\Delta u = f \quad \text{with } f = 1.0$$

**Boundary Conditions**:
$$u = x_1 \quad \text{on } \partial\Omega$$

**Finite Element Space**:
- Lagrangian elements, order 1 (bilinear on quadrilaterals)
- $H^1$ conformity (continuous across elements)

**Weak Formulation**:
$$a(u,v) = \int_\Omega \nabla v \cdot \nabla u \, d\Omega$$
$$l(v) = \int_\Omega f \cdot v \, d\Omega$$

### Focus of This Tutorial

Unlike Tutorial 1, where we solved the complete problem and examined the assembled system, here we'll **pause at the integration step** and dissect what happens when we evaluate:

```julia
matcontrib = a(u_basis, v_basis)  # What happens inside here?
veccontrib = l(v_basis)            # And here?
```

The key question: **How does Gridap compute these integrals numerically?**

> **📘 Note**: If any of these setup commands are unfamiliar, please review Tutorial 1 for detailed explanations. This tutorial assumes you understand the basic Gridap workflow and focuses specifically on the numerical integration aspects.

In [49]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
using Gridap

  Activating project at `~/Progs/git_repos/CMOE/CMOE-Tutorials`
project at `~/Progs/git_repos/CMOE/CMOE-Tutorials`


## 1.2 Defining Geometry, Model, and Weak Form

We quickly set up the same problem from Tutorial 1 to have a foundation for exploring integration details.

### Creating the Computational Domain

```julia
L = 2.0                        # Domain size
n = 2                          # Number of cells per direction
domain = (0, L, 0, L)         # Square domain [0,2] × [0,2]
partition = (n, n)            # 2×2 mesh
model = CartesianDiscreteModel(domain, partition)
```

This creates a uniform $2 \times 2$ mesh with 4 square elements, each of size $1.0 \times 1.0$.

### Defining Finite Element Spaces

```julia
reffe = ReferenceFE(lagrangian, Float64, 1)  # Bilinear Lagrangian elements
V = FESpace(model, reffe; conformity=:H1, dirichlet_tags="boundary")
U = TrialFESpace(V, x->x[1])  # Trial space with BC: u = x₁ on boundary
```

- **Test space `V`**: Homogeneous boundary conditions ($v = 0$ on $\partial\Omega$)
- **Trial space `U`**: Non-homogeneous boundary conditions ($u = x_1$ on $\partial\Omega$)

### Setting Up Integration Measures

```julia
Ω = Triangulation(model)  # Domain triangulation
dΩ = Measure(Ω, 2)         # Integration measure with quadrature order 2
```

The `Measure` object encapsulates:
- **Quadrature rule**: Order 2 (exact for polynomials up to degree 3)
- **Integration domain**: The triangulation $\Omega$
- **Quadrature points and weights**: To be explored in this tutorial

### Defining the Weak Formulation

```julia
f(x) = 1.0                      # Constant source term
a(u,v) = ∫(∇(v)⋅∇(u))dΩ        # Bilinear form
l(v) = ∫(f*v)dΩ                # Linear form
```

**Key Observation**: The integration operator `∫(...)dΩ` is where all the numerical integration magic happens. In this tutorial, we'll unpack what occurs inside this operator.

> **📘 Note**: For detailed explanations of these setup steps, see Tutorial 1. Here we move quickly through setup to focus on integration mechanics.

In [50]:
L = 2.0
n = 2
domain = (0,L,0,L)
partition = (n,n)
model = CartesianDiscreteModel(domain,partition)
reffe = ReferenceFE(lagrangian,Float64,1)
V = FESpace(model,reffe;conformity=:H1,dirichlet_tags="boundary")
U = TrialFESpace(V,x->x[1])
Ω = Triangulation(model)
dΩ = Measure(Ω,2)
f(x) = 1.0
a(u,v) = ∫( ∇(v)⋅∇(u) )dΩ
l(v) = ∫( f*v )dΩ;

## 1.3 Obtaining Basis Functions

Now we extract the basis functions from our finite element spaces—these will be the building blocks for integration.

### Extracting Basis Functions

```julia
u_basis = get_trial_fe_basis(U)  # Trial space basis functions
v_basis = get_fe_basis(V)        # Test space basis functions
```

These functions return **cell-wise basis function objects**:

- **`u_basis`**: Contains the basis functions $\{\phi_j\}$ from the trial space $U$
  - Satisfies boundary conditions: $\phi_j = x_1$ on $\partial\Omega$ for boundary nodes
  - Free DoFs correspond to interior nodes
  
- **`v_basis`**: Contains the basis functions $\{\psi_i\}$ from the test space $V$
  - Homogeneous boundary conditions: $\psi_i = 0$ on $\partial\Omega$ for boundary nodes
  - Used to test the weak formulation

**Structure**:
Both are `CellField` objects that store:
- Reference element basis functions
- Transformation rules to physical elements
- Lazy evaluation mechanisms (computed on-demand)

### Why We Need Basis Functions

To evaluate the weak forms $a(u,v)$ and $l(v)$ numerically, we substitute basis functions:

$$K^e_{ij} = a(\phi_j, \psi_i) = \int_{\Omega_e} \nabla\psi_i \cdot \nabla\phi_j \, d\Omega$$

$$f^e_i = l(\psi_i) = \int_{\Omega_e} f \cdot \psi_i \, d\Omega$$

The basis functions provide the concrete functions to integrate.

> **📘 Note**: In Tutorial 1, we used these basis functions directly in `a(u_basis, v_basis)`. Here, we'll explore what happens step-by-step inside that evaluation.

In [51]:
u_basis = get_trial_fe_basis(U) # basis functions of the trial space
v_basis = get_fe_basis(V)   # basis functions of the test space

SingleFieldFEBasis():
 num_cells: 4
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 5177669196975850158

### Interesting Remark: Different Shapes for Trial and Test Basis Functions

When evaluating basis functions at a point, Gridap returns **different array shapes** for trial and test spaces. This is not a bug—it's a deliberate design choice that simplifies elemental matrix construction!

**Observation**:
```julia
ux = u_basis(Point(0.0, 0.0))  # Trial basis: returns shape [1, j]
vx = v_basis(Point(0.0, 0.0))  # Test basis: returns shape [i]
```

**Why Different Shapes?**

1. **Trial Basis Functions** (`u_basis`): Return a **row vector** (shape `[1, j]`)
   - Dimension: `1 × n_dof_per_cell`
   - For bilinear quads: `1 × 4`
   - Represents: $[\phi_1, \phi_2, \phi_3, \phi_4]$

2. **Test Basis Functions** (`v_basis`): Return a **column vector** (shape `[i]`)
   - Dimension: `n_dof_per_cell`
   - For bilinear quads: `4`
   - Represents: $[\psi_1, \psi_2, \psi_3, \psi_4]^T$

**The Clever Design: Automatic Matrix Construction**

This shape convention directly facilitates **outer product** computation for elemental matrices!

**Mathematical Perspective**:

The elemental stiffness matrix has entries:
$$K^e_{ij} = \int_{\Omega_e} \psi_i \cdot \phi_j \, d\Omega$$

At each quadrature point, we need to compute:
$$\psi_i \cdot \phi_j \quad \text{for all } i, j$$

This is an **outer product** that produces a matrix!

**With Gridap's Shape Convention**:

```julia
# At a quadrature point:
vx = [ψ₁, ψ₂, ψ₃, ψ₄]         # Column vector (shape: 4)
ux = [φ₁, φ₂, φ₃, φ₄]         # Row vector (shape: 1×4)

# Outer product automatically gives matrix:
vx * ux = [ψ₁, ψ₂, ψ₃, ψ₄] * [φ₁, φ₂, φ₃, φ₄]
        = [ψ₁φ₁  ψ₁φ₂  ψ₁φ₃  ψ₁φ₄
           ψ₂φ₁  ψ₂φ₂  ψ₂φ₃  ψ₂φ₄
           ψ₃φ₁  ψ₃φ₂  ψ₃φ₃  ψ₃φ₄
           ψ₄φ₁  ψ₄φ₂  ψ₄φ₃  ψ₄φ₄]  (4×4 matrix)
```

**For Gradient-Based Integrands** (like our Laplacian):

```julia
∇vx = [∇ψ₁, ∇ψ₂, ∇ψ₃, ∇ψ₄]     # Each ∇ψᵢ is a 2D vector
∇ux = [∇φ₁, ∇φ₂, ∇φ₃, ∇φ₄]     # Each ∇φⱼ is a 2D vector

# Dot product of gradients:
(∇vx) ⋅ (∇ux) = [(∇ψ₁·∇φ₁)  (∇ψ₁·∇φ₂)  (∇ψ₁·∇φ₃)  (∇ψ₁·∇φ₄)
                 (∇ψ₂·∇φ₁)  (∇ψ₂·∇φ₂)  (∇ψ₂·∇φ₃)  (∇ψ₂·∇φ₄)
                 (∇ψ₃·∇φ₁)  (∇ψ₃·∇φ₂)  (∇ψ₃·∇φ₃)  (∇ψ₃·∇φ₄)
                 (∇ψ₄·∇φ₁)  (∇ψ₄·∇φ₂)  (∇ψ₄·∇φ₃)  (∇ψ₄·∇φ₄)]  (4×4 matrix)
```

**This is exactly the integrand for the elemental stiffness matrix!**

**Benefits of This Convention**:

1. **Natural Matrix Construction**:
   - No need to explicitly loop over indices $i, j$
   - Outer product is implicit through broadcasting
   - One operation produces entire elemental matrix integrand

2. **Efficient Vectorization**:
   - Julia's broadcasting handles the outer product efficiently
   - Takes advantage of BLAS/LAPACK for matrix operations
   - Minimal memory allocations

3. **Clean Code**:
   - Expression `∇(v_basis) ⋅ ∇(u_basis)` naturally produces matrix-valued function
   - Mirrors mathematical notation
   - No manual index management

4. **Automatic Generalization**:
   - Works for any number of DoFs per element
   - Scales to 3D, different element types
   - No code changes needed for different basis function counts

**Verification**:

Let's check the shapes explicitly:

```julia
ux = u_basis(Point(0.0, 0.0))  # Trial basis
vx = v_basis(Point(0.0, 0.0))  # Test basis

println("Trial basis shape: ", size(ux))     # Expect: (1, 4)
println("Test basis shape: ", size(vx))      # Expect: (4,)
println("Outer product shape: ", size(vx * ux))  # Expect: (4, 4)
```

**Conceptual Summary**:

| Basis Type | Shape | Purpose | Matrix Role |
|------------|-------|---------|-------------|
| **Test** `v_basis` | Column `[i]` | Rows of elemental matrix | Test functions $\psi_i$ |
| **Trial** `u_basis` | Row `[1, j]` | Columns of elemental matrix | Trial functions $\phi_j$ |
| **Outer Product** | Matrix `[i, j]` | Complete elemental matrix | All $\psi_i \cdot \phi_j$ pairs |

**Mathematical Intuition**:

In finite elements, we compute:
$$K^e_{ij} = a(\phi_j, \psi_i) = \int_{\Omega_e} \text{(some operator on } \psi_i \text{ and } \phi_j)$$

The $i$ index runs over test functions (rows), and the $j$ index runs over trial functions (columns). By making test functions "column-like" and trial functions "row-like," the natural broadcasting/outer product in Julia automatically produces the $i \times j$ matrix we need!

**Why This Matters for Integration**:

When we evaluate `object(x)` where `object = ∇(v_basis) ⋅ ∇(u_basis)`:
- At each quadrature point $\mathbf{x}_q$
- We get a **4×4 matrix** of integrand values $(∇\psi_i \cdot ∇\phi_j)(\mathbf{x}_q)$
- These are ready to be multiplied by weights and Jacobians
- Then summed to produce the elemental stiffness matrix

**No explicit double loop needed!** The shape convention makes matrix construction implicit.

> **📘 Note**: This is a beautiful example of how thoughtful API design (choosing appropriate array shapes) can make numerical algorithms both more efficient and more readable. The code structure mirrors the mathematical structure of the problem.

In [77]:
ux = u_basis(Point(0.0,0.0))
vx = v_basis(Point(0.0,0.0))
@show size(ux)
@show size(vx)
@show vx*ux;


size(ux) = (1, 4)
size(vx) = (4,)
vx * ux = [1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]


---

# Part 2: Quadrature and Evaluation

Now we enter the heart of numerical integration. In this part, we'll evaluate the weak forms with basis functions to create domain contributions, then explore how Gridap handles quadrature points, evaluates integrands, and manages different domain styles.

---

## 2.1 Evaluating Weak Forms: From Expressions to Domain Contributions

The first step in numerical integration is evaluating the weak form expressions with basis functions.

### Computing Matrix and Vector Contributions

```julia
matcontrib = a(u_basis, v_basis)  # Matrix contributions
veccontrib = l(v_basis)            # Vector contributions
```

**What Happens Here?**

When we write `a(u_basis, v_basis)`:
1. Gridap substitutes basis functions into the bilinear form: $a(\phi_j, \psi_i)$
2. Creates the integrand expression: $\nabla\psi_i \cdot \nabla\phi_j$
3. Associates this with the integration measure `dΩ`
4. Returns a `DomainContribution` object

Similarly, `l(v_basis)` creates the integrand $f \cdot \psi_i$.

### Understanding DomainContribution

Recall from Tutorial 1 that `DomainContribution` is a dictionary mapping triangulations to integration data:

```
DomainContribution
└── dict: {Triangulation Ω => LazyArray of integrands}
```

**Inspecting the Contents**:

```julia
first(matcontrib.dict)  # Shows: (Ω, lazy_array_of_gradients)
first(veccontrib.dict)  # Shows: (Ω, lazy_array_of_source_terms)
```

The **value** in the dictionary is not the final integrated result—it's a **lazy representation** of:
- The integrand expression ($\nabla\psi_i \cdot \nabla\phi_j$ or $f \cdot \psi_i$)
- How to evaluate it (at quadrature points)
- How to transform it (Jacobians, weights)

**Key Insight**: At this stage, **no numerical integration has occurred yet**. We have instructions for integration, not results.

### The Two-Stage Process

Gridap separates integration into two stages:

1. **Expression Stage** (what we just did):
   - Build the integrand expression
   - Associate it with a domain and measure
   - Store as `DomainContribution`

2. **Integration Stage** (what we'll explore next):
   - Evaluate integrand at quadrature points
   - Apply quadrature weights
   - Apply Jacobian transformations
   - Sum to get elemental matrices/vectors

This separation allows Gridap to:
- Optimize evaluation order
- Reuse computations
- Support lazy evaluation strategies

> **📘 Note**: The `DomainContribution` object is like a recipe—it contains all the instructions for integration but hasn't performed the cooking yet.

In [52]:
matcontrib = a(u_basis,v_basis)
veccontrib = l(v_basis)

DomainContribution()

In [53]:
first(matcontrib.dict)

BodyFittedTriangulation() => Fill([0.6666666666666664 -0.16666666666666663 -0.16666666666666663 -0.33333333333333315; -0.16666666666666663 0.6666666666666664 -0.33333333333333315 -0.1666666666666666; -0.16666666666666663 -0.33333333333333315 0.6666666666666664 -0.1666666666666666; -0.33333333333333315 -0.1666666666666666 -0.1666666666666666 0.6666666666666663], 4)

**Important Observation**: When we inspect `first(matcontrib.dict)`, the output shows **computed values** immediately. This might seem to contradict our claim about lazy evaluation, but there's an important distinction to understand.

**What We See**:

The result is of type `Fill`, which displays actual numerical values:
```
Fill([[matrix entries...]], 4)
```

This indicates:
- **One unique matrix** is stored
- **Replicated 4 times** (once per cell)
- **Values are displayed** because they're already computed

**Why Are Values Already Computed?**

For our **uniform Cartesian mesh** with **constant coefficients**:
1. All 4 cells have **identical geometry** (same size, shape)
2. The **elemental stiffness matrix is identical** for all cells
3. Gridap recognizes this **homogeneity** and uses a `Fill` array

**What is a Fill Array?**

`Fill(value, n)` is an efficient data structure that:
- **Stores only one copy** of `value`
- **Logically represents** `n` copies
- **Immediately computes** the single value (since it's needed once)
- **Memory efficient**: O(1) storage instead of O(n)

**Example**:
```julia
Fill([1.0, 2.0, 3.0], 100)  # Stores one array, represents 100 copies
# Memory: one array [1.0, 2.0, 3.0]
# Logical size: 100 identical arrays
```

**In Our Case**:

```julia
Fill([elemental_matrix], 4)
```
- Stores: One 4×4 elemental stiffness matrix
- Represents: 4 copies (one per cell)
- Why: All cells have identical elemental matrices

**Contrast with LazyArray**

In a **more general setting** (non-uniform mesh, varying coefficients, curved elements), the result would be a `LazyArray`:

```julia
LazyArray{...}(computation_instructions)
```

**LazyArray Characteristics**:
1. **Defers computation**: Values not computed until accessed
2. **Different per cell**: Each cell may have different elemental matrix
3. **Memory efficient**: Stores computation recipe, not results
4. **Computed on-demand**: `lazy_array[i]` triggers computation for cell `i`

**When Would We See LazyArray?**

- **Non-uniform meshes**: Elements with different sizes/shapes
- **Spatially varying coefficients**: Material properties changing by location
- **Curved elements**: Non-affine isoparametric mappings
- **Nonlinear problems**: Integrand depends on solution values

**Example Scenarios**:

1. **Current Case (Fill)**:
   ```julia
   # Uniform 2×2 mesh, constant coefficients
   matcontrib.dict[Ω] → Fill([K_e], 4)  # One matrix, 4 cells
   ```

2. **Non-uniform Mesh (LazyArray)**:
   ```julia
   # Varying element sizes: 1×1, 1×2, 2×1, 2×2
   matcontrib.dict[Ω] → LazyArray(...)  # Different K_e per cell
   ```

3. **Spatially Varying Coefficient (LazyArray)**:
   ```julia
   # Diffusion coefficient κ(x,y)
   a(u,v) = ∫(κ*∇v⋅∇u)dΩ
   matcontrib.dict[Ω] → LazyArray(...)  # κ varies by location
   ```

**Lazy Evaluation in Action**:

For a `LazyArray`, values are computed only when accessed:

```julia
lazy_array = LazyArray(computation_recipe)
# No computation yet!

K_e_cell_3 = lazy_array[3]  
# NOW computation happens for cell 3 only
```

**Performance Implications**:

| Array Type | Storage | Computation | Best For |
|------------|---------|-------------|----------|
| **Fill** | O(1) | Once (immediate) | Homogeneous problems |
| **LazyArray** | O(1) recipe | On-demand per cell | Heterogeneous problems |
| **Regular Array** | O(n) | All upfront | Small n, frequent access |

**Why This Design is Clever**:

1. **Automatic optimization**: Gridap detects homogeneity and uses `Fill`
2. **No user intervention**: Happens transparently
3. **Memory efficient**: Both `Fill` and `LazyArray` avoid storing redundant data
4. **Computation efficient**: `Fill` computes once, `LazyArray` computes only what's needed

**Inspecting the Type**:

```julia
result = first(matcontrib.dict)[2]  # Get the array part

if result isa Fill
    println("Homogeneous: One elemental matrix for all cells")
    println("Computed immediately and replicated")
elseif result isa LazyArray
    println("Heterogeneous: Different elemental matrices")
    println("Computed on-demand when accessed")
end
```

**Key Takeaway**:

- **Fill arrays** show computed values because there's only one unique value to compute
- **LazyArrays** would defer computation until indexing
- Both are **lazy in spirit**: they avoid unnecessary work
- Gridap **automatically chooses** the most efficient representation

**For This Tutorial**:

Our uniform mesh produces `Fill` arrays, so we see computed values immediately. This is actually **beneficial for learning**—we can inspect the elemental matrices directly! In production codes with complex geometries, you'd see `LazyArray` instead, and values would compute as needed during assembly.

> **📘 Note**: The distinction between `Fill` and `LazyArray` is an implementation detail that Gridap handles automatically. Users write the same high-level code regardless of which is used—the library optimizes behind the scenes based on problem structure.

### Inspecting Vector Contributions

Let's examine the structure of the vector contribution as well:

In [54]:
first(veccontrib.dict)

BodyFittedTriangulation() => [[0.2499999999999999, 0.24999999999999986, 0.2499999999999999, 0.2499999999999999], [0.2499999999999999, 0.24999999999999986, 0.2499999999999999, 0.2499999999999999], [0.2499999999999999, 0.24999999999999986, 0.2499999999999999, 0.2499999999999999], [0.2499999999999999, 0.24999999999999986, 0.2499999999999999, 0.2499999999999999]]

## 2.2 Understanding the `integrate` Function

> **📘 Note**: By manually calling `integrate`, we can inspect intermediate steps and understand exactly how numerical integration works in Gridap.

Now we'll dive into the core integration function that transforms integrands into numerical values.

The next sections will explore each of these steps in detail.

### The Integration Pipeline

$$K^e_{ij} = \int_{\Omega_e} (\nabla\psi_i \cdot \nabla\phi_j) \, d\Omega \approx \sum_{q=1}^{n_q} (\nabla\psi_i \cdot \nabla\phi_j)(\mathbf{x}_q) \, w_q \, |\mathbf{J}(\boldsymbol{\xi}_q)|$$

When we evaluated `a(u_basis, v_basis)`, Gridap created a `DomainContribution` but **did not integrate yet**. The actual integration happens through the `integrate` function.

**Mathematical Formula**:

### Manual Integration

5. **Sum**: Combine all contributions: $\sum_q (\cdots) w_q |\mathbf{J}|$

Let's manually perform what happens inside `a(u_basis, v_basis)`:4. **Apply Jacobian**: Include $|\mathbf{J}(\boldsymbol{\xi}_q)|$ for coordinate transformation

3. **Apply Weights**: Multiply by quadrature weights $w_q$

```julia2. **Evaluate Integrand**: Compute `object(x_q)` at each quadrature point

object = ∇(u_basis)⋅∇(v_basis)  # Create the integrand expression1. **Extract Quadrature Points**: Get $\{\mathbf{x}_q\}$ and weights $\{w_q\}$

cell_mats = integrate(object, dΩ.quad)  # Perform numerical integration

```The `integrate(object, quad)` function performs these operations:



**Step-by-Step Breakdown**:### What `integrate` Does Internally



1. **Create Integrand Object**:   - Produces same structure as `a(u_basis, v_basis)`

   ```julia   - Associates integrated matrices with triangulation

   object = ∇(u_basis)⋅∇(v_basis)   - Creates a new `DomainContribution`

   ```   ```

   - Computes gradients of trial and test basis functions   add_contribution!(matcontrib2, dΩ.quad.trian, cell_mats)

   - Takes dot product: $\nabla\phi_j \cdot \nabla\psi_i$   matcontrib2 = DomainContribution()

   - Returns a lazy cell field (not yet evaluated)   ```julia

3. **Package as DomainContribution** (optional):

2. **Integrate Using Quadrature**:

   ```julia   - Returns integrated elemental matrices

   cell_mats = integrate(object, dΩ.quad)   - Performs numerical integration over each cell

   ```   - `dΩ.quad`: The quadrature object from our measure

In [55]:
using Gridap.CellData: DomainContribution, add_contribution!
object  =  ∇(u_basis)⋅∇(v_basis)
cell_mats = integrate(object,dΩ.quad)
matcontrib2 = DomainContribution()
add_contribution!(matcontrib2,dΩ.quad.trian,cell_mats)

DomainContribution()

a(u,v) is the same as integrate(object,quad)

In [56]:
using Gridap.CellData: get_domains
get_contribution(matcontrib, first(get_domains(matcontrib))) == get_contribution(matcontrib2, first(get_domains(matcontrib2)))

true

Let's explore integrate function:

get quadrature points

In [57]:
quad = dΩ.quad
x = get_cell_points(quad)

CellPoint()

let's explore what's in x

In [58]:
using Gridap.CellData: CellDatum, get_data
@show isa(x,CellDatum)
@show get_data(x);
@show DomainStyle(x);

x isa CellDatum = true
get_data(x) = Fill(VectorValue{2, Float64}[(0.21132486540518708, 0.21132486540518708), (0.7886751345948129, 0.21132486540518708), (0.21132486540518708, 0.7886751345948129), (0.7886751345948129, 0.7886751345948129)], 4)
DomainStyle(x) = ReferenceDomain()


**Understanding the Output**:

> **📘 Note**: Gauss quadrature points are NOT uniformly spaced—they're strategically placed to maximize integration accuracy for polynomial integrands.

1. **`isa(x, CellDatum)`**: Confirms `x` is a cell-wise data structure

   - Not a simple array, but a lazy evaluation object- **Weights**: All equal to $w = 0.25$ (sum to 1.0 over unit square)

   - Data is computed per-cell as needed  - Evaluates to: $\xi \approx 0.2113$ and $\xi \approx 0.7887$

  - $\xi = \frac{1}{2} \pm \frac{1}{2\sqrt{3}}$

2. **`get_data(x)`**: Shows the underlying array structure- **Point locations**: Gauss-Legendre points shifted to $[0,1]$:

   - One entry per cell (4 cells in our mesh)- **Number of points**: $n_q = 2 \times 2 = 4$ per cell

   - Each entry contains coordinates of quadrature pointsFor **2×2 Gauss quadrature** (order 2):

   - Format depends on whether points are in reference or physical domain

**Why These Specific Values?**

3. **`DomainStyle(x)`**: Indicates the coordinate system

   - `ReferenceDomain()`: Points in reference element $[0,1]^2$   - `PhysicalDomain()`: Points mapped to physical elements

## 3.2 Evaluating the Integrand at Quadrature Points

Once we have quadrature points, the next step is evaluating the integrand expression at those points.

In [59]:
bx = object(x)

4-element Fill{Array{Float64, 3}}, with entries equal to [1.2440169358562925 -0.4553418012614796 -0.4553418012614796 -0.33333333333333326; 0.6666666666666667 -0.45534180126147955 0.12200846792814618 -0.3333333333333333; 0.6666666666666667 0.12200846792814618 -0.45534180126147955 -0.3333333333333333; 0.08931639747704091 0.12200846792814624 0.12200846792814624 -0.33333333333333337;;; -0.4553418012614796 0.6666666666666666 -0.33333333333333326 0.1220084679281462; -0.45534180126147955 1.2440169358562925 -0.3333333333333333 -0.4553418012614796; 0.12200846792814618 0.0893163974770409 -0.3333333333333333 0.12200846792814625; 0.12200846792814624 0.6666666666666667 -0.33333333333333337 -0.45534180126147955;;; -0.4553418012614796 -0.33333333333333326 0.6666666666666666 0.1220084679281462; 0.12200846792814618 -0.3333333333333333 0.0893163974770409 0.12200846792814625; -0.45534180126147955 -0.3333333333333333 1.2440169358562925 -0.4553418012614796; 0.12200846792814624 -0.33333333333333337 0.666666

explain why evaluating object at point x gives the previous result

Explore what is the domain style of the quadrature data

In [60]:
quad.data_domain_style

ReferenceDomain()

explore what is the domain style of integration

In [61]:
quad.integration_domain_style


PhysicalDomain()

For the case where data is in reference element and integration at physical domain, we apply the following

In [62]:
@show cell_map = get_cell_map(quad.trian)
@show cell_Jt = lazy_map(∇,cell_map)
@show cell_Jtx = lazy_map(evaluate,cell_Jt,quad.cell_point);

cell_map = get_cell_map(quad.trian) = Gridap.Fields.AffineField{2, 2, Float64, 4}[AffineField() AffineField(); AffineField() AffineField()]
cell_Jt = lazy_map(∇, cell_map) = Fill(ConstantField(), 2, 2)
cell_Jtx = lazy_map(evaluate, cell_Jt, quad.cell_point) = Fill(TensorValue{2, 2, Float64, 4}[(1.0, 0.0, 0.0, 1.0), (1.0, 0.0, 0.0, 1.0), (1.0, 0.0, 0.0, 1.0), (1.0, 0.0, 0.0, 1.0)], 4)


In [63]:
Apply lazy_map function to get the values

UndefVarError: UndefVarError: `Apply` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [64]:
using Gridap.Fields: IntegrationMap
lazy_map(IntegrationMap(),bx,quad.cell_weight,cell_Jtx)

4-element Fill{Matrix{Float64}}, with entries equal to [0.6666666666666664 -0.16666666666666663 -0.16666666666666663 -0.33333333333333315; -0.16666666666666663 0.6666666666666664 -0.33333333333333315 -0.1666666666666666; -0.16666666666666663 -0.33333333333333315 0.6666666666666664 -0.1666666666666666; -0.33333333333333315 -0.1666666666666666 -0.1666666666666666 0.6666666666666663]